In [109]:
import numpy as np
import pandas as pd

In [110]:
data=pd.read_csv("/content/data.csv")
output= open("Output.txt",'w')
data.head()

,Position ID,Position Status,Time,Time Out,Timecard Hours (as Time),Pay Cycle Start Date,Pay Cycle End Date,Employee Name,File Number
0,WFS000054,Active,NaN,NaN,NaN,NaN,NaN,"SiWgh, PraGhjEM",54
1,WFS000065,Active,09/12/2023 10:08 AM,09/12/2023 01:53 PM,3:45,09/10/2023,09/23/2023,"REsaXiaWE, XAis",65
2,WFS000065,Active,09/12/2023 02:23 PM,09/12/2023 07:02 PM,4:39,09/10/2023,09/23/2023,"REsaXiaWE, XAis",65
3,WFS000065,Active,09/13/2023 10:08 AM,09/13/2023 02:20 PM,4:12,09/10/2023,09/23/2023,"REsaXiaWE, XAis",65
4,WFS000065,Active,09/13/2023 02:50 PM,09/13/2023 08:44 PM,5:54,09/10/2023,09/23/2023,"REsaXiaWE, XAis",65


In [111]:
# It is assumed that each row refers to a single Employee (commans should be removed)
data['Employee Name']

0                   SiWgh, PraGhjEM
1                   REsaXiaWE, XAis
2                   REsaXiaWE, XAis
3                   REsaXiaWE, XAis
4                   REsaXiaWE, XAis
                   ...             
1479                WgAyeW, RayCEWd
1480                WgAyeW, RayCEWd
1481                WgAyeW, RayCEWd
1482                WgAyeW, RayCEWd
1483    ArveXE RECerE, AWdres JesAs
Name: Employee Name, Length: 1484, dtype: object

In [112]:
def func(name):
    # Function to removed commmas from name and capitalize 1st letter or each word
    name= name.replace(",","").title()
    return name

data['Employee Name'] = data['Employee Name'].apply(func)
data['Employee Name']

0                   Siwgh Praghjem
1                   Resaxiawe Xais
2                   Resaxiawe Xais
3                   Resaxiawe Xais
4                   Resaxiawe Xais
                   ...            
1479                Wgayew Raycewd
1480                Wgayew Raycewd
1481                Wgayew Raycewd
1482                Wgayew Raycewd
1483    Arvexe Recere Awdres Jesas
Name: Employee Name, Length: 1484, dtype: object

In [113]:
len(data['Employee Name'].unique())

106

### Employees who has worked for 7 consecutive days-
We will be considering the day as 'worked' based on the "Time" column which is expected to the Time of start of a shift

In [114]:
print("Name and Position of Employees who have worked for 7 consecutive days:", file=output )
n,m= data.shape
i=0
worked_for_7_consecutive_days= [] # desired output
while i<n: # Looping over all rows
    name= data['Employee Name'][i]
    position= data['Position ID'][i]
    streak=1
    if data['Time'][i] is not np.nan:
        prev= int(data['Time'][i].split("/")[1])
    else:
        prev= None
    i += 1
    while i<n and data['Employee Name'][i]==name: # Looping over one Employee
        if data['Time'][i] is np.nan:
            i += 1
            continue
        this = int(data['Time'][i].split("/")[1])
        if this==prev:
            i += 1
            continue
        if this==prev+1:
            streak += 1
            if streak==7:
                worked_for_7_consecutive_days.append((name,position))
                print(name," , PositionID=",position, file= output)
                while i<n and data['Employee Name'][i]==name:
                    # Since this employee is already added to our list, we skip rows with his/her name
                    i += 1
                break
        else:
            # streak breaks if this != prev+1
            streak=1
        i += 1
        prev= this
print("---------------------------------------------------------------------------------------", file=output)

In [115]:
worked_for_7_consecutive_days

[('Sparks Kewwemh', 'WFS000101'),
 ('Siwgh Jaspreem', 'WFS000424'),
 ('Cereira Jr Jese', 'WFS000456'),
 ('Gaewdia Jaaw Carxes', 'WFS000473')]

### Employees who have less than 10 hours of time between shifts but greater than 1 hour

In [116]:
def time_diff(t1,t2):
    # Function to calculate the time difference(in hours) and check whether it fulfils our condition
    if t1 is np.nan or t2 is np.nan:
        return False

    date1, time1, p1= t1.split(" ")
    date2, time2, p2= t2.split(" ")
    hour1,min1= (time1.split(":"))
    hour2, min2= time2.split(":")
    hour1, min1, hour2, min2 = int(hour1), int(min1), int(hour2), int(min2)

    if p1=="PM":
        hour1 += 12
    if p2=="PM":
        hour2 += 12

    day_diff= int(date1.split("/")[1])-int(date2.split("/")[1])
    if day_diff>=2: return False
    if day_diff==0:
        net_diff= hour2-hour1
    else:
        net_diff= 24-hour1 + hour2
    if net_diff>1 and net_diff<10:
        return True
    return False


print("Name and Position of employees who have less than 10 hours of time between shifts but greater than 1 hour:", file=output )
time_between_shifts = []
i= 0
while i<n:
    # We calculate the difference b/w "Time" of current row and "Time Out" of previous row
    name= data['Employee Name'][i]
    position= data['Position ID'][i]
    prev_tout= data['Time Out'][i]
    i += 1
    while i<n and data['Employee Name'][i]==name:
        if  time_diff(prev_tout, data['Time'][i]):
            time_between_shifts.append((name, position))
            print(name," , PositionID=",position, file= output)
            while i<n and data['Employee Name'][i]==name:
                i += 1
            break
        prev_tout= data['Time Out'][i]
        i += 1
print("---------------------------------------------------------------------------------------", file=output)
time_between_shifts

[('Camaxawe Ceghaw', 'WFS000153'),
 ('Xee Xacar', 'WFS000200'),
 ('Payaw Dawiex', 'WFS000368'),
 ('Cereira Jr Jese', 'WFS000456'),
 ('Perkiws Marrawce', 'WFS000469'),
 ('Cemma Jessica', 'WFS000503'),
 ('Caragixxa Awmhewy', 'WFS000514'),
 ('Dexgadixxe Redarme Chrismiaw S', 'WFS000523'),
 ('Hacixmew Deawme Devew', 'WFS000536')]


### Employees who has worked for more than 14 hours in a single shift-

In [117]:
print("Name and position of employees who have worked for more than 14 hours in a single shift:", file=output)
worked_for_14hrs_shift= []
i=0
while i<n:
    time= data['Timecard Hours (as Time)'][i]
    name= data['Employee Name'][i]
    position= data['Position ID'][i]
    if time is np.nan or name is np.nan:
        i += 1
        continue
    time= int(time.split(":")[0])
    if time>=14:
        worked_for_14hrs_shift.append((name, position))
        print(name," , PositionID=",position, file= output)
        while i<n and data['Employee Name'][i]==name:
            i += 1
        continue
    i += 1
print("---------------------------------------------------------------------------------------", file=output)
worked_for_14hrs_shift

[('Dexgadixxe Redarme Chrismiaw S', 'WFS000523')]